In [8]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 1️⃣ 필수 패키지 설치
!pip install streamlit pandas plotly
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [9]:
# 2️⃣ CSV 파일 복사 (Colab 내부에서 사용 가능하게)
import os

file_path = "/mnt/data/ad_simulation_data (3).csv"
local_file_path = "ad_data_0222.csv"
os.system(f"cp {file_path} {local_file_path}")


512

In [10]:


# 3️⃣ Streamlit 앱 코드 저장
streamlit_code = """
import streamlit as st
import pandas as pd
import plotly.express as px
from datetime import datetime

# CSV 파일 로드
file_path = "ad_data_0222.csv"
df = pd.read_csv(file_path)

# Streamlit 페이지 설정
st.set_page_config(page_title='이지혜_광고 효과 분석 대시보드', layout='wide')

# 제목 및 설명
st.title('광고 효과 분석 대시보드')
st.write('광고 캠페인의 성과를 분석하고, 광고 지출 대비 매출 효과를 평가합니다.')

# 광고 채널 선택 필터
selected_channel = st.multiselect('분석할 광고 채널을 선택하세요:', options=df['광고_채널'].unique(), default=df['광고_채널'].unique())
filtered_df = df[df['광고_채널'].isin(selected_channel)]


# 광고_시작일 및 광고_종료일 컬럼을 datetime 형식으로 변환
filtered_df['광고_시작일'] = pd.to_datetime(filtered_df['광고_시작일'])
filtered_df['광고_종료일'] = pd.to_datetime(filtered_df['광고_종료일'])

# 사용자로부터 분석 기간 입력 받기
start_date, end_date = st.date_input(
    '분석 기간 선택',
    [filtered_df['광고_시작일'].min(), filtered_df['광고_종료일'].max()]
)

# datetime.date -> datetime.datetime 변환
start_date = datetime.combine(start_date, datetime.min.time())
end_date = datetime.combine(end_date, datetime.max.time())

# 선택한 기간과 광고 기간이 겹치는 데이터 필터링
filtered_df = filtered_df[
    (filtered_df['광고_시작일'] <= end_date) &  # 광고 시작일이 선택한 종료일 이전
    (filtered_df['광고_종료일'] >= start_date)  # 광고 종료일이 선택한 시작일 이후
]


# 데이터 다운로드 버튼
st.subheader('데이터 다운로드')
csv = filtered_df.to_csv(index=False).encode('utf-8-sig')
st.download_button(
    label="CSV 파일 다운로드",
    data=csv,
    file_name='filtered_data.csv',
    mime='text/csv'
)


#   예측모델추가

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# 데이터 준비
X = filtered_df[['광고_지출액']]
y = filtered_df['매출액']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 학습
model = LinearRegression()
model.fit(X_train, y_train)

# 예측 결과 시각화
filtered_df['매출 예측'] = model.predict(X)
fig8 = px.scatter(filtered_df, x='광고_지출액', y='매출액', title='광고 지출액과 매출액 예측', trendline='ols')
st.plotly_chart(fig8, use_container_width=True)

# 추가적인 KPI분석
# ROAS 계산
filtered_df['ROAS'] = filtered_df['매출액'] / filtered_df['광고_지출액']
st.subheader('광고 채널별 ROAS 분석')
roas_channel = filtered_df.groupby('광고_채널')['ROAS'].mean().reset_index()
fig6 = px.bar(roas_channel, x='광고_채널', y='ROAS', title='광고 채널별 ROAS', color='광고_채널')
st.plotly_chart(fig6, use_container_width=True)


# 광고 채널별 광고 지출액 비교
st.subheader('광고 채널별 광고 지출 비교')
channel_spending = filtered_df.groupby('광고_채널')['광고_지출액'].sum().reset_index()
fig1 = px.bar(channel_spending, x='광고_채널', y='광고_지출액', title='광고 채널별 광고 지출액', color='광고_채널')
st.plotly_chart(fig1, use_container_width=True)

# 광고 형식별 클릭률(CTR) 분석
st.subheader('광고 형식별 클릭률(CTR)')
filtered_df['CTR'] = (filtered_df['광고_클릭_횟수'] / filtered_df['광고_노출_횟수']) * 100
format_ctr = filtered_df.groupby('광고_형식')['CTR'].mean().reset_index()
fig2 = px.bar(format_ctr, x='광고_형식', y='CTR', title='광고 형식별 평균 클릭률', color='광고_형식')
st.plotly_chart(fig2, use_container_width=True)

# 거주 지역별 매출액 분포
st.subheader('지역별 매출 분포')
region_sales = filtered_df.groupby('거주_지역')['매출액'].sum().reset_index()
fig3 = px.pie(region_sales, names='거주_지역', values='매출액', title='지역별 매출 비율')
st.plotly_chart(fig3, use_container_width=True)

# 연령대별 평균 매출액 분석
st.subheader('연령대별 평균 매출액')
filtered_df['연령대'] = filtered_df['연령'].apply(lambda x: f"{(x//10)*10}대")
age_sales = filtered_df.groupby('연령대')['매출액'].mean().reset_index()
fig4 = px.line(age_sales, x='연령대', y='매출액', title='연령대별 평균 매출액', markers=True)
st.plotly_chart(fig4, use_container_width=True)

# 광고 회상률 및 태도 변화 분석
st.subheader('광고 회상률 & 태도 변화 분석')
fig5 = px.scatter(filtered_df, x='광고_회상률', y='광고_태도_변화', title='광고 회상률과 태도 변화의 관계', color='광고_채널', size='광고_지출액')
st.plotly_chart(fig5, use_container_width=True)

# 원본 데이터 표시
st.subheader('원본 데이터 보기')
st.dataframe(filtered_df)

"""

In [11]:
# 4️⃣ Streamlit 앱 코드 저장
with open("app.py", "w") as f:
    f.write(streamlit_code)

In [ ]:
# 5️⃣ Streamlit 실행 & Cloudflare Tunnel 사용
!streamlit run app.py &>/dev/null &
!./cloudflared tunnel --url http://localhost:8501 --no-autoupdate

2025-03-08T00:38:24Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-03-08T00:38:24Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-03-08T00:38:27Z INF +--------------------------------------------------------------------------------------------+
2025-03-08T00:38:27Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-03-08T00:38:27Z INF |  https://partnership-degrees-representative-opposed.tr